In [1]:
import pandas as pd
import numpy as np
import re
import warnings
import datetime as dt
warnings.filterwarnings('ignore')
path="../../../ukb/data/"

In [2]:
year=2021
month=1
day=1

config = dict(time_points = ['0'])

In [3]:
def read_data(file,path=path,chunksize=10000):
    df_chunk = pd.read_csv('%s%s' % (path,file), chunksize=chunksize, iterator=True,sep=',')#,nrows=1000)
    df = pd.concat(df_chunk, ignore_index=True)
    return df

#create independent variable set
def createindepvars(df,tps=config['time_points']):
    cols=np.asarray(df.columns)
    colselected=np.asarray([])
    timeperiod=np.asarray([])
    newcolnames=np.asarray([])

    for i,s in enumerate(cols):

        t=s[len(s)-6:len(s)]
        u=s[len(s)-2:len(s)]
        if u=="_0":
            colselected=np.append(colselected,s)
            inds = [i for i,c in enumerate(s) if c=='_']
            t=s[inds[len(inds)-2]+1:inds[len(inds)-1]]
            u=s[0:inds[len(inds)-2]]
            timeperiod=np.append(timeperiod,t)
            newcolnames=np.append(newcolnames,u)

    df_out=pd.DataFrame([])

    #for i in range(int(max(timeperiod))):
    for i in tps:
        indices=[j for j,c in enumerate(timeperiod) if c==str(i)]
        dfa=df[colselected[indices]]

        #print(colselected[indices])
        dfa.columns=newcolnames[indices]
        dfa['eid']=df['eid']
        dfa['time_point']=str(i)
        df_out=pd.concat([df_out,dfa],axis=0)
    print(df_out.shape)
    return df_out

def returndata(df,tp,minvals=50000):
    df=df[(df['time_point']==tp)]
    cols=[col for col in df.columns if df[col].count()>minvals]
    df=df[cols]
    return df

In [4]:
genotype=read_data("ukb_genotype.csv")
ukb=read_data("all_fields_251020.csv")
customfields=read_data("ukb_customfields.csv")
death_codes=read_data("hm_cod_update_02sep2020.csv")

In [5]:
customfields=pd.merge(customfields,genotype,on='eid',how='left')
customfields=customfields[['eid','parental_ad_status','APOE4_Carriers','APOE4_assigned','Genotype']]

death_codes=death_codes[['n_eid','s_40000_0_0']]
death_codes['death']=1
death_codes.columns=['eid','death_date','death']

ukb_tp0=createindepvars(ukb,tps=config['time_points'])

ukb_tp0=pd.merge(ukb_tp0,death_codes,on='eid',how='left')

ukb_temp=pd.merge(ukb_tp0[['eid']],customfields,on='eid',how='left')
ukb_tp0['parental_ad_status']=ukb_temp['parental_ad_status']
ukb_tp0['APOE4_Carriers']=ukb_temp['APOE4_Carriers']
ukb_tp0['Genotype']=ukb_temp['Genotype']

ukb_tp0['Age_Today']=ukb_tp0['age_when_attended_assessment_centre_f21003']+\
(dt.datetime(year, month, day)-pd.to_datetime(ukb_tp0['date_of_attending_assessment_centre_f53'])).dt.days/365


(502523, 1516)


In [6]:
ukb_tp0.to_pickle('%s%s' % (path,'ukb_tp0_.p'))